# Troublesome Tiles

The naive approach of using np.interp to match adjacent full-sensor image tiles, treating them as grayscale images, works surprisingly well.  But it fails miserably in some cases.

One of the worst failures is `pano_188_3_668275957.301_NAVCAM_RIGHT`.  It is assembled from these tile images:

```
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_01_0LLJ (0, 0, 1288, 968)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_02_0LLJ (1272, 0, 1296, 968)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_03_0LLJ (2552, 0, 1296, 968)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_04_0LLJ (3832, 0, 1288, 968)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_05_0LLJ (0, 952, 1288, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_06_0LLJ (1272, 952, 1296, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_07_0LLJ (2552, 952, 1296, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_08_0LLJ (3832, 952, 1288, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_09_0LLJ (0, 1912, 1288, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_10_0LLJ (1272, 1912, 1296, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_11_0LLJ (2552, 1912, 1296, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_12_0LLJ (3832, 1912, 1288, 976)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_13_0LLJ (0, 2872, 1288, 968)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_14_0LLJ (1272, 2872, 1296, 968)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_15_0LLJ (2552, 2872, 1296, 968)
Tile NRE_0015_0668275956_042ECM_N0030188NCAM00400_16_0LLJ (3832, 2872, 1288, 968)
```

I've cached a copy of it here.

![pano_188...](./images/pano_188_3_668275957.301_NAVCAM_RIGHT.png)

Why does this mapping fail so miserably?

In [9]:
%matplotlib widget

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from skimage import color, io
from skimage.util import img_as_ubyte

from band_finder.image_db import ImageDB
from band_finder.image_cache import ImageCache
from band_finder.tile_matcher import TileMatcher
from band_finder.bayer_to_rgb import bayer_to_rgb

examples_dir = Path("../examples").resolve()

db_path = list(examples_dir.glob("*.db"))[0]
assert db_path.exists()

image_db = ImageDB(db_path)

cache_dir = examples_dir / "image_cache"
assert cache_dir.exists()

image_cache = ImageCache(image_db, cache_dir)

def get_rgb_image(image_id):
    image = image_cache.get_image(image_id)
    # If it is a bayer image, convert it to RGB.
    if image_id[2:3] == "E":
        image = bayer_to_rgb(image)
    return image
    
def get_lab_image(image_id):        
    return color.rgb2lab(get_rgb_image(image_id))

class TileMatcherBuilder:
    def __init__(self, get_image=get_lab_image):
        self._get_image = get_image
        self._matcher = TileMatcher()
        # Original, unadjusted images:
        self._image_ids = []
        
    def add_tile(self, image_id, bbox):
        self._image_ids.append(image_id)

        image = self._get_image(image_id)
        origin = bbox[:2]
        print("Add", image_id, "at", origin)

        self._matcher.add(image, origin)

    def add_test_tiles(self):
        add = self.add_tile
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_01_0LLJ", (0, 0, 1288, 968))
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_02_0LLJ", (1272, 0, 1296, 968))
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_03_0LLJ", (2552, 0, 1296, 968))
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_04_0LLJ", (3832, 0, 1288, 968))
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_05_0LLJ", (0, 952, 1288, 976))
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_06_0LLJ", (1272, 952, 1296, 976))
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_07_0LLJ", (2552, 952, 1296, 976))
        add("NRE_0015_0668275956_042ECM_N0030188NCAM00400_08_0LLJ", (3832, 952, 1288, 976))
        
    def get_composite(self):
        return self._matcher.composite()
    
    def show_originals(self):
        num_images = len(self._image_ids)
        num_cols = 4
        num_rows = num_images // num_cols
        if num_rows * num_cols < num_images:
            num_rows += 1
        fig, axes = plt.subplots(num_rows, num_cols, squeeze=False)
        fig.set_tight_layout(True)
        ix = iy = 0
        for image_id in self._image_ids:
            image = get_rgb_image(image_id)
            ax = axes[iy][ix]
            ax.set_axis_off()
            ax.imshow(image)
            ix += 1
            if ix >= num_cols:
                ix = 0
                iy += 1


Let's try to re-tile a portion of the image.

In [10]:
builder = TileMatcherBuilder()
builder.add_test_tiles()
builder.show_originals()


Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_01_0LLJ at (0, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_02_0LLJ at (1272, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_03_0LLJ at (2552, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_04_0LLJ at (3832, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_05_0LLJ at (0, 952)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_06_0LLJ at (1272, 952)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_07_0LLJ at (2552, 952)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_08_0LLJ at (3832, 952)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# Ensure the lab values are in range.
def rescale_channel(image, channel, min_valid, max_valid):
    curr_values = image[:, :, channel]
    max_in = np.max(curr_values)
    min_in = np.min(curr_values)
    print(f"Rescale: channel {channel} is {min_in}...{max_in}.")
    
    if (max_in > max_valid) or (min_in < min_valid):
        d_in = max_in - min_in
        d_out = max_valid - min_valid
        scale = d_out / d_in
        new_values = (curr_values - min_in) * scale + min_valid
        image[:, :, channel] = new_values
        
        print(f"Rescale: Adjusted {channel} to {np.min(new_values)}...{np.max(new_values)}.")

def clip_channel(image, channel, min_valid, max_valid):
    curr_values = image[:, :, channel]
    curr_values[curr_values > max_valid] = max_valid
    curr_values[curr_values < min_valid] = min_valid
    image[:, :, channel] = curr_values

def rescale_lab(image):
    rescale_channel(image, 0, 0.0, 100.0)
    rescale_channel(image, 1, -1.0, 1.0)
    rescale_channel(image, 2, -1.0, 1.0)
    

lab_image_data = builder.get_composite()
rescale_lab(lab_image_data)

fig, axes = plt.subplots(1, 1)
axes.imshow(img_as_ubyte(color.lab2rgb(lab_image_data)))


Rescale: channel 0 is 0.0...98.16679382324219.
Rescale: channel 1 is -17.54530906677246...22.40850067138672.
Rescale: Adjusted 1 to -1.0...1.0.
Rescale: channel 2 is 0.0...43.117774963378906.
Rescale: Adjusted 2 to -1.0...1.0.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Blegh.**  Do things look any better if I stick to RGB throughout?

In [4]:
rgb_builder = TileMatcherBuilder(get_image=get_rgb_image)
rgb_builder.add_test_tiles()
rgb_image_data = builder.get_composite()

def rescale_rgb(image):
    for channel, cmin, cmax in [
        [0, 0, 255],
        [1, 0, 255],
        [2, 0, 255]
    ]:
        rescale_channel(image, channel, cmin, cmax)

rescale_rgb(rgb_image_data)
# TileMatcher always produces composites with float64 values.
rgb_fig, rgb_axes = plt.subplots(1, 1)
rgb_axes.imshow(img_as_ubyte(rgb_image_data.astype(np.uint8)))

Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_01_0LLJ at (0, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_02_0LLJ at (1272, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_03_0LLJ at (2552, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_04_0LLJ at (3832, 0)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_05_0LLJ at (0, 952)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_06_0LLJ at (1272, 952)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_07_0LLJ at (2552, 952)
Add NRE_0015_0668275956_042ECM_N0030188NCAM00400_08_0LLJ at (3832, 952)
Rescale: channel 0 is 0.0...98.16679382324219.
Rescale: channel 1 is -17.54530906677246...22.40850067138672.
Rescale: Adjusted 1 to 0.0...255.00001525878906.
Rescale: channel 2 is 0.0...43.117774963378906.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Nope.

I think the problem in this case is that the first column of the second row is an incomplete tile image.